In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime

import pymysql

In [23]:
com = re.compile("(20\d{2}-\d{2}-\d{2} \d{2}_\d{2}_\d{2}\.xlsx)$")

sent_dir = os.path.join(NEW_ORDER_SAVE_DIR, '已发')
sent_bk_dir = os.path.join(WORK_DIR,'备份')

for file in os.listdir(sent_dir):
    result = com.search(file)
    if result:
        print(file)
        data = pd.read_excel(os.path.join(sent_dir,file))
        order_data = pd.read_excel(os.path.join(EXPORT_DIR,"订单 %s" %(result.group())))
in_mysql_df = order_data[order_data['订单号'].isin(data['订单号'].unique())]

date_fileds = ['导出订单时间','下单时间','支付时间']
for field in date_fileds:
    in_mysql_df[field] = in_mysql_df[field].apply(lambda x:x.strftime('%Y-%m-%d %H:%M:%S'))

三金_新订单 2019-12-12 15_37_45.xlsx


In [24]:
conn = pymysql.connect(**MYSQL_MALL_DIC)
cursor = conn.cursor()



fields = ["订单号","发货商","数量","支付金额","备注","收件人","联系方式","收货地址","goods_type",
          "导出订单时间"  ,'下单时间', '支付时间']
date_time_len = 3


sql = 'insert into order_details(%s) values(%s,%s)' %(','.join(fields),",".join(np.repeat('%s',len(fields)-date_time_len)),
        ",".join(np.repeat("str_to_date(%s,'%%Y-%%m-%%d %%H:%%i:%%S')",date_time_len)))


in_mysql_df = in_mysql_df[fields]

in_mysql_df = in_mysql_df.fillna(value={"备注":""})
for index,row in in_mysql_df.iterrows():
    print(row.tolist())
    cursor.execute(sql,row.tolist())
conn.commit()
conn.close()

[7801733718286446, '三金', 1, 0, '', '郑丽君', 13786171008, '湖南省长沙市天心区桂花坪街道园艺路兴汝金城2期14栋2503', 'free', '2019-12-12 15:37:45', '2019-12-12 14:12:16', '2019-12-12 14:12:22']
[19121251101531, '三金', 1, 0, '', '张雅淇', 15576667890, '湖南省长沙市岳麓区望兴西西里13栋小卖部', 'free', '2019-12-12 15:37:45', '2019-12-12 14:12:31', '2019-12-12 14:12:40']
[19121248494810, '三金', 1, 0, '', '于菲菲', 18007316578, '湖南省长沙市开福区汐园庭院餐厅', 'free', '2019-12-12 15:37:45', '2019-12-12 13:12:32', '2019-12-12 13:12:43']
[19121253985148, '三金', 1, 0, '', '黎雅杰', 13975187591, '湖南省长沙市芙蓉区望龙小区A25栋三单元501号', 'free', '2019-12-12 15:37:45', '2019-12-12 12:12:09', '2019-12-12 12:12:18']
[19121249485597, '三金', 1, 0, '', '李小姐', 18007483266, '湖南省长沙市天心区回龙山巷回龙公寓b栋1002', 'free', '2019-12-12 15:37:45', '2019-12-12 12:12:25', '2019-12-12 12:12:39']
[19121252575210, '三金', 1, 0, '', '彭松梅', 18684819641, '湖南省长沙市岳麓区中海国际社区4期', 'free', '2019-12-12 15:37:45', '2019-12-12 12:12:40', '2019-12-12 12:12:53']
[19121257485710, '三金', 1, 0, '', '万威', 18975281371, '湖南省湘潭市湘乡市文华苑